# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Decision-Tree" data-toc-modified-id="Decision-Tree-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Decision Tree</a></div><div class="lev2 toc-item"><a href="#Create-input-data" data-toc-modified-id="Create-input-data-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Create input data</a></div><div class="lev2 toc-item"><a href="#概念" data-toc-modified-id="概念-12"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>概念</a></div><div class="lev3 toc-item"><a href="#Example-5.2-计算系统的熵" data-toc-modified-id="Example-5.2-计算系统的熵-121"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Example 5.2 计算系统的熵</a></div><div class="lev3 toc-item"><a href="#Example-5.3-计算系统的增益熵，并且选择" data-toc-modified-id="Example-5.3-计算系统的增益熵，并且选择-122"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Example 5.3 计算系统的增益熵，并且选择</a></div><div class="lev4 toc-item"><a href="#算法-ID-3" data-toc-modified-id="算法-ID-3-1221"><span class="toc-item-num">1.2.2.1&nbsp;&nbsp;</span>算法 ID-3</a></div><div class="lev2 toc-item"><a href="#CART-算法" data-toc-modified-id="CART-算法-13"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>CART 算法</a></div><div class="lev1 toc-item"><a href="#Ignore-below" data-toc-modified-id="Ignore-below-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Ignore below</a></div>

# Decision Tree

## Create input data

In [121]:
import pandas as pd
import numpy as np

In [94]:
age = ['young'] * 5 + ['adult'] * 5 + ['old'] * 5
work = ['unemployed'] * 2 + ['employed'] * 2 + ['unemployed'] * 3 + ['employed'] + ['unemployed'] * 4 + ['employed'] * 2 + ['unemployed'] * 1
house = ['no'] * 3 + ['yes'] * 1 + ['no'] * 3 + ['yes'] * 5 + ['no'] * 3
credit = ['regular'] + ['good'] * 2 + ['regular'] * 3 + ['good'] * 2 + ['perfect'] * 3 + ['good'] * 2 + ['perfect'] + ['regular']
target = [0] * 2 + [1] * 2 + [0] * 3 + [1] * 7 + [0]
df = pd.DataFrame({'ID':range(1, len(target)+1),'age':age, 'work':work, 'house':house, 'credt':credit, 'Y':target})

In [95]:
df

,ID,age,work,house,credt,Y
0,1,young,unemployed,no,regular,0
1,2,young,unemployed,no,good,0
2,3,young,employed,no,good,1
3,4,young,employed,yes,regular,1
4,5,young,unemployed,no,regular,0
5,6,adult,unemployed,no,regular,0
6,7,adult,unemployed,no,good,0
7,8,adult,employed,yes,good,1
8,9,adult,unemployed,yes,perfect,1
9,10,adult,unemployed,yes,perfect,1


## 概念

Entropy

$H(p) = -\sum_{i=1}^n p_i \log_2 p_i$

Gini-index

$Gini(p) = \sum_{k=1}^K p_k(1-p_k) = 1-\sum_{k=1}^K p_k^2$

越大系统的有效信息越少，说明越混乱（无聊）

对于熵，我们定义：

Compute entropy of a system without any classification, using $D$ and $H(D)$, find all the target Y/N, use 

$H(D) = -\sum_{k=1}^K \frac{|C_k|}{|D|}\log_2 \frac{|C_k|}{|D|}$
where $K$ here indicates the classification or categories of our target

同时我们计算conditional entropy
$H(D|A) = \sum_{i=1}^n \frac{|D_i|}{|D|}H(D_i)$, 每一个$i$ 代表沿着某一个特征$A$的分类，同时定义

$f(D,A)= H(D) - H(D|A)$

因为系统的entropy代表稳定与否，这个差越大，说明分类越好

同时，对于基尼系数：

$Gini(D) = 1- \sum_{k=1}^K\Big( \frac{|C_i|}{|D|}\Big)^2$

$Gini(D,A=a) = \frac{|D_1|}{|D|} Gini(D_1) + \frac{|D_2|}{|D|} Gini(D_2) $

这里的$k$代表我们的target 里面的种类，$A$代表沿着特征的A分类，$D_1$代表 $A=a$的个数， $D_2$ 代表$A!=a$的个数

In [198]:
def H_D_condition_A(df, A, D='Y'):
    K_elements = df[A].unique()
    size_D = df.shape[0]
    ans = 0
    for i in K_elements:
        D_i = df.loc[df[A]==i]
        size_Di = D_i.shape[0]
        ans += size_Di/size_D * H_D(D_i)
    return ans

def H_D(df,D='Y'):
    df = df.copy()
    HD = df.groupby([D])[[D]].count()
    HD['tot'] = HD[D].sum()
    HD['c_k/D'] = HD[D]/HD['tot']
    HD['ln(c_k/D)'] = np.log2(HD['Y']/HD['tot'])
    HD['HD_k'] = - HD['c_k/D'] * HD['ln(c_k/D)']
    HD_scalar = HD['HD_k'].sum()
    return HD_scalar

def Gini_D(df, D='Y'):
    
    df = df.copy()
    Gini = df.groupby([D])[[D]].count()
    Gini['tot'] = Gini[D].sum()
    Gini['c_k/D'] = Gini[D]/Gini['tot']
    Gini['(c_k/D)^2'] = Gini['c_k/D'] * Gini['c_k/D']
    Gini_scalar = 1 - Gini['(c_k/D)^2'].sum()
    #print(Gini)
    return Gini_scalar

def g(df, A, D='Y'):
    return H_D(df) - H_D_condition_A(df, A)


def Gini_D_A(df, A, a, D='Y'):
    df_a = df.loc[df[A]==a]
    df_not_a = df.loc[df[A]!=a]
    return df_a.shape[0]/df.shape[0] * Gini_D(df_a, D) + df_not_a.shape[0]/df.shape[0] * Gini_D(df_not_a, D)
    

### Example 5.2 计算系统的熵

In [199]:
for col in df.columns:
    if col=='ID':
        continue
    print(col, "{:.3f}".format(g(df, col)))

assert round(g(df, 'age'),3) == 0.083
#age 0.083
#work 0.324
#house 0.420
#credt 0.363
#Y 0.971

age 0.083
work 0.324
house 0.420
credt 0.363
Y 0.971


### Example 5.3 计算系统的增益熵，并且选择

#### 算法 ID-3

这个例子说明，当我们有很多的feature 时，怎么判断他们的顺序，那就是 g(增益熵）越大的分类最新利用；
接着下一步分类

Here we look at the third feature $A_3$, and since the corresponding $Y$ under $A_3='yes'$ or $D_1$ is all $1$, thus we ignore it and only look at $D_2=0$.

In [200]:
# if we select house is 'no'
# then, we look at other features
A2 = 'house'
print("Y=", df.loc[df[A2]=='yes']['Y'].unique()) # --> this subset always gives Y=1, ignore it. Look at 'no'
for col in df.columns:
    if col in ['ID', 'house', 'Y']:
        continue
    df_temp = df.loc[df['house']=='no']
    print(col, "{:.3f}".format(g(df_temp, col)))

#Y= [1]
#age 0.252
#work 0.918
#credt 0.474

Y= [1]
age 0.252
work 0.918
credt 0.474


## CART 算法

In [212]:
# A_1, age

for count, A in enumerate(df.columns):
    print("*******")
    if A in ['ID', 'Y']:
        continue

    uniques = df[A].unique()
    for a in uniques:

        gini = Gini_D_A(df, A, a) # note, gini needs to first select attribute, then value of this attribute
        print("Gini(D, A_{}, {}={}) = {:.3f}".format(count, A, a, gini))

*******
*******
Gini(D, A_1, age=young) = 0.440
Gini(D, A_1, age=adult) = 0.480
Gini(D, A_1, age=old) = 0.440
*******
Gini(D, A_2, work=unemployed) = 0.320
Gini(D, A_2, work=employed) = 0.320
*******
Gini(D, A_3, house=no) = 0.267
Gini(D, A_3, house=yes) = 0.267
*******
Gini(D, A_4, credt=regular) = 0.320
Gini(D, A_4, credt=good) = 0.474
Gini(D, A_4, credt=perfect) = 0.364
*******


In [176]:
Gini_D_A(df, 'age')

0.9362962962962962

# Ignore below

In [132]:
import operator
from math import log

def splitDataSet(dataSet,axis,value):
    """
    按照给定特征划分数据集
    :param axis:划分数据集的特征的维度
    :param value:特征的值
    :return: 符合该特征的所有实例（并且自动移除掉这维特征）
    """

    # 循环遍历dataSet中的每一行数据
    retDataSet = []
    # 找寻 axis下某个特征的非空子集
    for featVec in dataSet:
        if featVec[axis] == value:
            reduceFeatVec = featVec[:axis] # 删除这一维特征
            reduceFeatVec.extend(featVec[axis+1:])
            retDataSet.append(reduceFeatVec)
    return retDataSet

# 计算的始终是类别标签的不确定度
def calcShannonEnt(dataSet):
    """
    计算训练数据集中的Y随机变量的香农熵
    :param dataSet:
    :return:
    """
    numEntries = len(dataSet) # 实例的个数
    labelCounts = {}
    for featVec in dataSet: # 遍历每个实例，统计标签的频次
        currentLabel = featVec[-1] # 表示最后一列
        # 当前标签不在labelCounts map中，就让labelCounts加入该标签
        if currentLabel not in labelCounts.keys(): labelCounts[currentLabel] =0
        labelCounts[currentLabel] +=1


    shannonEnt = 0.0
    for key in labelCounts:
        prob = float(labelCounts[key]) / numEntries
        shannonEnt -= prob * log(prob,2) # log base 2
    return shannonEnt


def calcConditionalEntropy(dataSet,i,featList,uniqueVals):
    """
    计算x_i给定的条件下，Y的条件熵
    :param dataSet: 数据集
    :param i: 维度i
    :param featList: 数据集特征列表
    :param unqiueVals: 数据集特征集合
    :return: 条件熵
    """
    ce = 0.0
    for value in uniqueVals:
        subDataSet = splitDataSet(dataSet,i,value)
        prob = len(subDataSet) / float(len(dataSet)) # 极大似然估计概率
        ce += prob * calcShannonEnt(subDataSet) #∑pH(Y|X=xi) 条件熵的计算 
    return ce

# 计算信息增益
def calcInformationGain(dataSet,baseEntropy,i):
    """
    计算信息增益
    :param dataSet: 数据集
    :param baseEntropy: 数据集中Y的信息熵
    :param i: 特征维度i
    :return: 特征i对数据集的信息增益g(dataSet | X_i)
    """
    featList = [example[i] for example in dataSet] # 第i维特征列表
    uniqueVals = set(featList) # 换成集合 - 集合中的每个元素不重复
    newEntropy = calcConditionalEntropy(dataSet,i,featList,uniqueVals)
    infoGain = baseEntropy - newEntropy # 信息增益
    return infoGain



def majorityCnt(classList):
    """
    返回出现次数最多的分类名称
    :param classList: 类列表
    :retrun: 出现次数最多的类名称
    """

    classCount = {}
    for vote in classList:
        if vote not in classCount.keys(): classCount[vote] = 0
        classCount[vote] +=1
    sortedClassCount = sorted(classCount.items(),key = operator.itemgetter(1),reverse = True)
    return sortedClassCount[0][0]

def chooseBestFeatureToSplitByID3(dataSet):
    """
    选择最好的数据集划分
    :param dataSet:
    :return:
    """
    numFeatures = len(dataSet[0]) -1 # 最后一列是分类
    baseEntropy = calcShannonEnt(dataSet)
    bestInfoGain = 0.0
    bestFeature = -1
    for i in range(numFeatures): # 遍历所有维度特征
        infoGain = calcInformationGain(dataSet,baseEntropy,i)
        if(infoGain > bestInfoGain):
            bestInfoGain = infoGain
            bestFeature = i
    return bestFeature # 返回最佳特征对应的维度


def calcInformationGainRate(dataSet,baseEntropy,i):
        """
        计算信息增益比
        :param dataSet: 数据集
        :param baseEntropy: 数据集中Y的信息熵
        :param i: 特征维度i
        :return: 特征i对数据集的信息增益g(dataSet|X_i)
        """
        numEntries = len(dataSet)
        labelCounts = {}
        for featVec in dataSet:
                currentLabel = featVec[i]
                if currentLabel not in labelCounts.keys(): labelCounts[currentLabel] =0
                labelCounts[currentLabel] +=1
        shannonEnt = 0.0
        for key in labelCounts:
                prob = float(labelCounts[key]) / numEntries
                shannonEnt -= prob * log(prob,2)

        return calcInformationGain(dataSet,baseEntropy,i) / shannonEnt

def chooseBestFeatureToSplitByC45(dataSet):
        """
        选择最好的数据集划分方式
        :param dataSet:
        :return:
        """
        numFeatures = len(dataSet[0]) -1 # 最后一列是分类
        baseEntropy = calcShannonEnt(dataSet)
        bestInfoGainRate =0.0
        bestFeature = -1
        for i in range(numFeatures):
                infoGainRate = calcInformationGainRate(dataSet,baseEntropy,i)
                if (infoGainRate > bestInfoGainRate):
                        bestInfoGainRate = infoGainRate
                        bestFeature = i
        return bestFeature


def noInformationGainToSplitByID3(dataSet):
        """
        不使用信息增益概念，而是直接判断条件熵的大小
        """
        numFeatures = len(dataSet[0]) -1
        bestConditionEntropy = 1.0
        bestFeature =-1
        for i in range(numFeatures):
                featList = [example[i] for example in dataSet]
                uniqueVals = set(featList)
                conditionEntropy = calcConditionalEntropy(dataSet,i,featList,uniqueVals)
                if (conditionEntropy < bestConditionEntropy):
                        bestConditionEntropy = conditionEntropy
                        bestFeature =i
        return bestFeature


def createTree(dataSet,labels,chooseBestFeatureToSplitFunc = chooseBestFeatureToSplitByID3):
    """
    创建决策树
    :param dataSet: 数据集
    :param labels: 数据集每一维的名称
    :return: 决策树
    """
    classList = [example[-1] for example in dataSet] # 类别列表
    if classList.count(classList[0]) == len(classList): # 统计属于列别classList[0]的个数
        return classList[0] # 当类别完全相同则停止继续划分
    if len(dataSet[0]) ==1: # 当只有一个特征的时候，遍历所有实例返回出现次数最多的类别
        return majorityCnt(classList) # 返回类别标签
    bestFeat = chooseBestFeatureToSplitFunc(dataSet)
    bestFeatLabel = labels[bestFeat]
    myTree ={bestFeatLabel:{}}  # map 结构，且key为featureLabel
    del (labels[bestFeat])
    # 找到需要分类的特征子集
    featValues = [example[bestFeat] for example in dataSet]
    uniqueVals = set(featValues)
    for value in uniqueVals:
        subLabels = labels[:] # 复制操作
        myTree[bestFeatLabel][value] = createTree(splitDataSet(dataSet,bestFeat,value),subLabels)
    return myTree

# change map feature to label's index
def mapFeatureToLabelIndex(map,labels):
        for key in map.keys():
                for i in range(len(labels)):
                        if key == labels[i]:
                                return key,i


# 决策树预测函数
def predict(testData,decisionTree,labels):
        # 获得决策树结点的下标
        feature_label,feature_index = mapFeatureToLabelIndex(decisionTree,labels)
        tree = decisionTree[feature_label][testData[feature_index]]
        # 判断该树是叶子结点还是子结点
        if (~isinstance(tree,dict)): # 如果是叶子结点，则直接返回结果
                return tree
        else: # 子结点则继续递归
                return predict(testData,tree,labels)

# 决策树准确率判断
def calPrecision(dataSet,predictSet):
        length = len(dataSet)
        count = 0
        for i in range(length):
              if dataSet[i][-1] == predictSet[i]:
                        count +=1
        return count / length *100


In [133]:
def createDataSet():
    """
    创建数据集
    """
    dataSet = [[u'1000',u'青年', u'否', u'否', u'一般', u'拒绝'],
                [u'2000',u'青年', u'否', u'否', u'好', u'拒绝'],
                [u'7000',u'青年', u'是', u'否', u'好', u'同意'],
                [u'7100',u'青年', u'是', u'是', u'一般', u'同意'],
                [u'3000',u'青年', u'否', u'否', u'一般', u'拒绝'],
                [u'3500',u'中年', u'否', u'否', u'一般', u'拒绝'],
                [u'3600',u'中年', u'否', u'否', u'好', u'拒绝'],
                [u'8000',u'中年', u'是', u'是', u'好', u'同意'],
                [u'9000',u'中年', u'否', u'是', u'非常好', u'同意'],
                [u'9200',u'中年', u'否', u'是', u'非常好', u'同意'],
                [u'8600',u'老年', u'否', u'是', u'非常好', u'同意'],
                [u'7800',u'老年', u'否', u'是', u'好', u'同意'],
                [u'10000',u'老年', u'是', u'否', u'好', u'同意'],
                [u'6500',u'老年', u'是', u'否', u'非常好', u'同意'],
                [u'3000',u'老年', u'否', u'否', u'一般', u'拒绝'],
                ]
    labels = [u'工资',u'年龄', u'有工作', u'有房子', u'信贷情况']
    predict_labels = [u'工资',u'年龄', u'有工作', u'有房子', u'信贷情况']
    # 返回数据集和每个维度的名称
    return dataSet, labels,predict_labels


In [134]:
# 计算数据集的基尼指数
def calcGini(dataSet):
    numEntries = len(dataSet)
    labelCounts ={}
    # 给所有可能分类创建字典
    for featVec in dataSet:
        currentLabel = featVec[-1]
        if currentLabel not in labelCounts.keys():
            labelCounts[currentLabel] =0
        labelCounts[currentLabel]+=1
    Gini =1.0
    for key in labelCounts:
        prob = float(labelCounts[key])/numEntries
        Gini -= prob * prob
    return Gini


In [135]:
def splitDataSet(dataSet,axis,value):
    """
    按照给定特征划分数据集
    :param axis:划分数据集的特征的维度
    :param value:特征的值
    :return: 符合该特征的所有实例（并且自动移除掉这维特征）
    """

    # 循环遍历dataSet中的每一行数据
    retDataSet = []
    # 找寻 axis下某个特征的非空子集
    for featVec in dataSet:
        if featVec[axis] == value:
            reduceFeatVec = featVec[:axis] # 删除这一维特征
            reduceFeatVec.extend(featVec[axis+1:])
            retDataSet.append(reduceFeatVec)
    return retDataSet


In [136]:
def splitOtherDataSetByValue(dataSet,axis,value):
        """
    按照给定特征划分数据集
    :param axis:划分数据集的特征的维度
    :param value:特征的值
    :return: 不符合该特征的所有实例（并且自动移除掉这维特征）
    """
    # 循环遍历dataSet中的每一行数据
        retDataSet = []
    # 找寻 axis下某个特征的非空子集
        for featVec in dataSet:
                if featVec[axis] != value:
                        reduceFeatVec = featVec[:axis] # 删除这一维特征
                        reduceFeatVec.extend(featVec[axis+1:])
                        retDataSet.append(reduceFeatVec)
        return retDataSet


In [137]:
def majorityCnt(classList):
    """
    返回出现次数最多的分类名称
    :param classList: 类列表
    :retrun: 出现次数最多的类名称
    """

    classCount = {}
    for vote in classList:
        if vote not in classCount.keys(): classCount[vote] = 0
        classCount[vote] +=1
    sortedClassCount = sorted(classCount.items(),key = operator.itemgetter(1),reverse = True)
    return sortedClassCount[0][0]


In [138]:
def binaryZationDataSet(bestFeature,bestSplitValue,dataSet):
        # 求特征标签数
        featList = [example[bestFeature] for example in dataSet]
        uniqueValues = set(featList)

        # 特征标签输超过2，对数据集进行二值划分 为了看出决策树构造时的区别，这里特征标签为2时也进行处理
        if len(uniqueValues) >= 2:
                for i in range(len(dataSet)):
                        if dataSet[i][bestFeature] == bestSplitValue: # 不做处理
                                pass
                        else:
                                dataSet[i][bestFeature] = '其他' 


In [139]:
def chooseBestFeatureToSplitByCART(dataSet):
    numFeatures = len(dataSet[0]) -1
    bestGiniIndex = 1000000.0
    bestSplictValue =[]
    bestFeature = -1
    # 计算Gini指数
    for i in range(numFeatures):
        featList = [example[i] for example in dataSet]
        # 这里只针对离散变量 & 特征标签
        uniqueVals = set(featList)
        bestGiniCut = 1000000.0
        bestGiniCutValue =[]
        Gini_value =0.0
        # 计算在该特征下每种划分的基尼指数，并且用字典记录当前特征的最佳划分点
        for value in uniqueVals:
            # 计算subDataSet的基尼指数
            subDataSet = splitDataSet(dataSet,i,value)
            prob = len(subDataSet) / float(len(dataSet))
            Gini_value = prob * calcGini(subDataSet)
            # 计算otherDataSet的基尼指数
            otherDataSet = splitOtherDataSetByValue(dataSet,i,value)
            prob = len(otherDataSet) / float(len(dataSet))
            Gini_value = Gini_value + prob * calcGini(otherDataSet)
            # 选择最优切分点
            if Gini_value < bestGiniCut:
                bestGiniCut = Gini_value
                bestGiniCutValue = value

        # 选择最优特征向量
        GiniIndex = bestGiniCut
        if GiniIndex < bestGiniIndex:
            bestGiniIndex = GiniIndex
            bestSplictValue = bestGiniCutValue
            bestFeature = i
            print(bestFeature,bestSplictValue)

    # 若当前结点的划分结点特征中的标签超过3个，则将其以之前记录的划分点为界进行二值化处理
    binaryZationDataSet(bestFeature,bestSplictValue,dataSet)
    return bestFeature


In [140]:
def createTree(dataSet,labels,chooseBestFeatureToSplitFunc = chooseBestFeatureToSplitByID3):
    """
    创建决策树
    :param dataSet: 数据集
    :param labels: 数据集每一维的名称
    :return: 决策树
    """
    classList = [example[-1] for example in dataSet] # 类别列表
    if classList.count(classList[0]) == len(classList): # 统计属于列别classList[0]的个数
        return classList[0] # 当类别完全相同则停止继续划分
    if len(dataSet[0]) ==1: # 当只有一个特征的时候，遍历所有实例返回出现次数最多的类别
        return majorityCnt(classList) # 返回类别标签
    bestFeat = chooseBestFeatureToSplitFunc(dataSet)
    bestFeatLabel = labels[bestFeat]
    myTree ={bestFeatLabel:{}}  # map 结构，且key为featureLabel
    del (labels[bestFeat])
    # 找到需要分类的特征子集
    featValues = [example[bestFeat] for example in dataSet]
    uniqueVals = set(featValues)
    for value in uniqueVals:
        subLabels = labels[:] # 复制操作
        myTree[bestFeatLabel][value] = createTree(splitDataSet(dataSet,bestFeat,value),subLabels,chooseBestFeatureToSplitFunc)
    return myTree

In [141]:
dataSet,labels,predict_labels= createDataSet()
myTree = createTree(dataSet,labels,chooseBestFeatureToSplitByCART)

0 3000
2 否
3 否
0 10000
1 老年
2 否


In [146]:
import treePlotter

mpl.rcParams['font.sans-serif'] = ['SimHei']  # 指定默认字体
mpl.rcParams['axes.unicode_minus'] = False  # 解决保存图像时负号'-'显示为方块的问题

#treePlotter.createPlot(myTree)

In [149]:
myTree

{'有房子': {'否': {'有工作': {'否': '拒绝', '其他': '同意'}}, '其他': '同意'}}